In [1]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle
from sklearn.cluster import DBSCAN
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import collections
import gensim
model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [2]:
f = open('../koyuu_noun_sample_list.binaryfile','rb')
sample_vec_list = pickle.load(f)

In [ ]:
f = open("../sample_relation_vec_fold/fact" + ".binaryfile",'rb')
relation_vec_list = pickle.load(f)
len(relation_vec_list)

In [ ]:
import random 

sample_relation_list= random.sample(relation_vec_list, int(len(relation_vec_list)/5))
len(sample_relation_list)

In [ ]:
# 密なクラスタを算出

eps_minPts = {}
minPts_data = {}

print("noun_vec_list数 : ", len(sample_noun_vec_list))

for eps in range(4,5):
    eps = eps/10
    
    for minPts in range(14,15):
        dbscan = DBSCAN(eps=eps,min_samples=minPts).fit(sample_noun_vec_list)
        
        y_dbscan = dbscan.labels_
        
        print("y_dbscan", y_dbscan)
        
        print("eps:",eps,",minPts:", minPts)
        
        # outlier数
        print("outlier数", len(np.where(y_dbscan ==-1)[0]))
        
        # クラスタ数
        print("クラスタ数", np.max(y_dbscan) +1)
        
        print("                            ")
        
        minPts_data[minPts] = y_dbscan
        eps_minPts[eps] = minPts_data
        
    # DBSCANの結果を保存
    f_name = "sample_relation_vec" + "_dic.binaryfile"
    #f_name = "hoge"
    save_at = "../dbscan_sample_koyuu_noun_fold/" + f_name
    print(save_at)
    f = open(save_at,'wb')
    pickle.dump(eps_minPts,f)
    f.close
    eps_minPts = {}
    minPts_data = {}

In [ ]:
f = open("../dbscan_sample_koyuu_noun_fold/sample_relation_vec_dic.binaryfile",'rb')
eps_minPts_dic = pickle.load(f)

In [ ]:
y_dbscan = eps_minPts_dic[0.4][14]

In [ ]:
# データ取得
#eps minPts n_class

mean_vec_list = []
            
n_class = np.max(y_dbscan) +1
n_outlier = len(np.where(y_dbscan == -1)[0])

for n in range(n_class+1):
    print("class", n)
    
    index = list(np.where(y_dbscan == n))
    index =list(index[0])
            
    d = np.array([sample_vec_list[i] for i in index])
        
    #クラスタの重心
    mean = np.mean(d, axis=0)
    mean_vec_list.append(mean)
        
                    

In [ ]:
len(mean_vec_list)

In [ ]:
# outlayerを追加
import random

index = list(np.where(y_dbscan == -1))
index =list(index[0])
d = np.array([sample_vec_list[i] for i in index])

hoge = random.sample(list(d), int(len(d)/5))

len(hoge)
mean_vec_list.extend(hoge)

In [ ]:
len(mean_vec_list)

In [ ]:
# 関係ベクトルを保存

f = open('../dbscan_sample_koyuu_noun_fold/sample_koyuu_relation_vec_list','wb')
pickle.dump(mean_vec_list, f)
f.close

In [ ]:
f = open("../dbscan_sample_koyuu_noun_fold/sample_koyuu_relation_vec_list",'rb')
fuga = pickle.load(f)
len(fuga)